## Despliegue del modelo de prediccion de Diabetes
--------------------------


Se hace el despliegue del mejor modelo de predicción encontrado, que en este caso se trata de un modelo de random forest. El cual sera cargado para ser utilizado por una api mediante MLflow y poder obtener prediciones.

## **Librerias**

In [23]:
import numpy as np
import pandas as pd
import joblib
import mlflow
import os
import requests
import mlflow.pyfunc
import re
np.random.seed(3)

## Funciones

-----------------------------

A continuacion se muestran unas funciones necesarias para el preprocesamiento de los datos de entrada y carga del modelo final.

In [2]:
def prep(x,columns):
    X = pd.DataFrame(x, columns=columns)
    bol = ['hypertension', 'heart_disease']
    num = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']
    cat = ['gender', 'smoking_history']
    X_bol = X[bol].values
    X_num = X[num].values
    X_cat = X[cat].values
    min_max = joblib.load('/home/kevo/Escritorio/mlds6/tdsp_template_grupo8/scripts/preprocessing/min_max_scaler.pkl')
    one_h = joblib.load('/home/kevo/Escritorio/mlds6/tdsp_template_grupo8/scripts/preprocessing/one_hencoder.pkl')
    X_num_minmax = min_max.transform(X_num)
    X_cat_onehot = one_h.transform(X_cat)
    X_fin = np.concatenate((X_num_minmax, X_bol ,X_cat_onehot),axis=1) 
    return X_fin.tolist()

In [3]:
def modelo():
    modelo = joblib.load('/home/kevo/Escritorio/proyectos/despliegue/proyec/RF_model.joblib')
    return modelo

Antes de desplegar el modelo, se configura todo el entorno de ML flow, primero se lanza un servidor web.

In [4]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

/home/kevo/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
2024/06/04 19:02:26 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/06/04 19:02:26 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759

se define el servidor que MLflow estará manejando que en nuestro caso se trata de un servidor local

In [5]:
mlflow.set_tracking_uri("http://localhost:5000")

Se crea un experimento de MLflow

In [6]:
exp_id = mlflow.create_experiment(name="diabetes", artifact_location="mlruns/")

y corremos el modelo en MLflow para que genere el archivo .pkl que es el se utilizara para generar las predicciones del despliegue

In [7]:
with mlflow.start_run(
        run_name="random_forest", experiment_id=exp_id
        ):
    model = modelo()
    mlflow.sklearn.log_model(model, "model")

se especifica la url del servidor de seguimiento de ml flow

In [8]:
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"

se lanza el api de MLflow 

In [9]:
command = """
mlflow models serve -m 'models:/diabetes/1' -p 8001 --env-manager 'local' &
"""
get_ipython().system_raw(command)

/home/kevo/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
2024/06/04 19:04:46 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/06/04 19:04:46 INFO mlflow.pyfunc.backend: === Running command 'exec gunicorn --timeout=60 -b 127.0.0.1:8001 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2024-06-04 19:04:46 -0500] [28841] [INFO] Starting gunicorn 22.0.0
[2024-06-04 19:04:46 -0500] [28841] [INFO] Listening at: http://127.0.0.1:8001 (28841)
[2024-06-04 19:04:46 -0500] [28841] [INFO] Using worker: sync
[2024-06-04 19:04:46 -0500] [28842] [INFO] Booting worker with pid: 28842


## Sistema de predicción de diabetes.
------------------
En esta sección se cargan los datos a introducir en el modelo de pronostico. Como primer paso se debe digitar la dirección donde se encuentra el archivo csv con los registros que ingresaran al modelo. A continuación, se ingresa la cantidad de registros a cargar y finalmente se puede ver una parte de ellos.

In [42]:
print('¡Hola! Bienvenido al sistema de detección de diabetes.\n')
dir = str(input("Ingresa una tabla de datos en formato csv: \n"))
df = pd.read_csv(dir)
n_registros = int(input("¿Cuantos registros quieres cargar?: \n"))
df = df[:n_registros]
columnas = df.columns
print('Revisa algunos de los registros cargados')
df.head(10)

¡Hola! Bienvenido al sistema de detección de diabetes.

Revisa algunos de los registros cargados


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
0,Female,51.0,0,0,current,36.77,6.6,159
1,Male,12.0,0,0,No Info,22.29,4.5,90
2,Female,60.0,1,0,former,34.24,6.2,90
3,Male,80.0,0,0,former,24.39,4.0,100
4,Male,80.0,0,1,No Info,35.00,4.5,145
5,Female,16.0,0,0,never,19.69,4.0,126
6,Male,78.0,1,0,never,22.14,3.5,80
7,Male,78.0,0,0,No Info,35.49,4.5,155
8,Female,49.0,0,0,never,31.28,6.2,145
9,Female,16.0,0,0,never,24.55,5.0,155


Se realiza el preprocesamiento de los registros cargados

In [43]:
df = df.values.tolist()
data = prep(df,columnas)

Y se hace un post con los datos preprocesados para que ingresen al servidor de MLflow, ejecuten el modelo de predicción y se obtengan los resultados correspondientes.

In [44]:
r = requests.post("http://localhost:8001/invocations", json={"inputs": data})
predictions = re.findall(r'\d+', r.text)
predictions = list(map(int, predictions))
for i in range(len(predictions)):
    print(f"Paciente {i+1}: {'No diabetes' if predictions[i]==0 else 'Diabetes'}")

Paciente 1: No diabetes
Paciente 2: No diabetes
Paciente 3: No diabetes
Paciente 4: No diabetes
Paciente 5: No diabetes
Paciente 6: No diabetes
Paciente 7: No diabetes
Paciente 8: No diabetes
Paciente 9: No diabetes
Paciente 10: No diabetes
Paciente 11: No diabetes
Paciente 12: No diabetes
Paciente 13: No diabetes
Paciente 14: No diabetes
Paciente 15: No diabetes
Paciente 16: No diabetes
Paciente 17: Diabetes
Paciente 18: No diabetes
Paciente 19: No diabetes
Paciente 20: No diabetes
Paciente 21: No diabetes
Paciente 22: No diabetes
Paciente 23: No diabetes
Paciente 24: No diabetes
Paciente 25: No diabetes
Paciente 26: No diabetes
Paciente 27: No diabetes
Paciente 28: No diabetes
Paciente 29: No diabetes
Paciente 30: No diabetes
Paciente 31: No diabetes
Paciente 32: No diabetes
Paciente 33: No diabetes
Paciente 34: No diabetes
Paciente 35: No diabetes
Paciente 36: No diabetes
Paciente 37: No diabetes
Paciente 38: No diabetes
Paciente 39: No diabetes
Paciente 40: No diabetes
Paciente 41: